In [1]:
suppressMessages(library(Signac))
suppressMessages(library(Seurat))
suppressMessages(library(GenomeInfoDb))
suppressMessages(library(ggplot2))
suppressMessages(library(RColorBrewer))

suppressMessages(library(tidyverse))
set.seed(1234)

suppressMessages(library(data.table))
#suppressMessages(library(JASPAR2020))
suppressMessages(library(TFBSTools))
suppressMessages(library(ChIPseeker))
suppressMessages(library(TxDb.Hsapiens.UCSC.hg38.knownGene))
suppressMessages(library(EnsDb.Hsapiens.v86))
#library(data.table)
#library(SeuratWrappers)
suppressMessages(library(ggalluvial))
suppressMessages(library(RColorBrewer))
suppressMessages(library(ggraph))
suppressMessages(library(igraph))
suppressMessages(library(readxl))

In [2]:
setwd('/diskmnt/Projects/snATAC_analysis/tumor_Alla/linkingGenes_v7.0/annotate.links/tumor_only_multiome/')
cancers = c('CRC', 'BRCA', 'HNSCC', 'CESC', 'OV', 'PDAC', 'UCEC', 'SKCM')

filter <- dplyr::filter
select <- dplyr::select

In [3]:
#use the folder where the output of Compute_links_to_genes_v7.0.R script is saved
#takes few minutes if the objects are large
all.links <- cancers %>% map(function(c) {
    obj <- readRDS(paste0('/diskmnt/Projects/snATAC_analysis/tumor_Alla/linkingGenes_v7.0/tumor_only_multiome/out/',c,'_atac.tumor_cells_multiome_obj.20230121.linked.rds'))
    links.tb <- Links(obj) %>% as.data.table() %>% mutate(Cancer = c)
    return(links.tb)
}) %>% rbindlist()

fwrite(all.links, 'All_links_tumor_only_multiome_8_cancers.tsv', sep='\t', row.names = FALSE)


## Open annotation files 

In [3]:
#load files
peakAnno <- fread('/diskmnt/Projects/snATAC_analysis/tumor_Alla/Annotate.peaks/v7.0/Annotation.EnsDb.Hsapiens.v86/Pancan_peaks_df_v7.0_EnsDb.Hsapiens.v86.annot.tsv')
peakAnno %>% head

genehancer <- fread('/diskmnt/Projects/snATAC_analysis/tumor_Alla/Annotate.peaks/v7.0/Peaks_overlapping_with_GeneHancer_enhancers_and_promoter_enhancers.txt')
epimap <- fread('/diskmnt/Projects/snATAC_analysis/tumor_Alla/Annotate.peaks/v7.0/Peaks_overlapping_with_EpiMap_promoters_enhancers.txt')
encode <- fread('/diskmnt/Projects/snATAC_analysis/tumor_Alla/Annotate.peaks/v7.0/Peaks_overlapping_with_ENCODE_CREs_SCREEN_annotated.txt')

all.links <- fread('/diskmnt/Projects/snATAC_analysis/tumor_Alla/linkingGenes_v7.0/annotate.links/tumor_only_multiome/All_links_tumor_only_multiome_8_cancers.tsv', header = TRUE)



seqnames,start,end,width,strand,annotation,geneChr,geneStart,geneEnd,geneLength,geneStrand,geneId,transcriptId,transcriptBiotype,distanceToTSS,ENTREZID,SYMBOL,GENENAME,peak.position,new_peak
<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>
chr1,2241786,2242286,501,*,"Intron (ENST00000378536/ENSG00000157933, intron 1 of 6)",3,2247996,2303376,55381,1,ENSG00000157933,ENST00000478223,processed_transcript,-5710,6497,SKI,SKI proto-oncogene,Intron,chr1-2241786-2242286
chr1,6833709,6834209,501,*,"Intron (ENST00000303635/ENSG00000171735, intron 3 of 22)",3,6834333,6834618,286,2,ENSG00000227950,ENST00000433407,processed_pseudogene,409,100270828,RPL37P9,ribosomal protein L37 pseudogene 9,Intron,chr1-6833709-6834209
chr1,59582266,59582766,501,*,"Intron (ENST00000424725/ENSG00000172456, intron 5 of 12)",3,59553918,59762728,208811,1,ENSG00000172456,ENST00000371210,protein_coding,28348,55277,FGGY,FGGY carbohydrate kinase domain containing,Intron,chr1-59582266-59582766
chr1,66545958,66546458,501,*,"Intron (ENST00000237247/ENSG00000118473, intron 2 of 26)",3,66534282,66748299,214018,1,ENSG00000118473,ENST00000371037,protein_coding,11676,84251,SGIP1,SH3GL interacting endocytic adaptor 1,Intron,chr1-66545958-66546458
chr1,72205471,72205971,501,*,"Intron (ENST00000357731/ENSG00000172260, intron 1 of 6)",3,72274552,72275159,608,1,ENSG00000233994,ENST00000445131,processed_pseudogene,-68581,100420259,GDI2P2,GDP dissociation inhibitor 2 pseudogene 2,Intron,chr1-72205471-72205971
chr1,72229785,72230285,501,*,"Intron (ENST00000357731/ENSG00000172260, intron 1 of 6)",3,72274552,72275159,608,1,ENSG00000233994,ENST00000445131,processed_pseudogene,-44267,100420259,GDI2P2,GDP dissociation inhibitor 2 pseudogene 2,Intron,chr1-72229785-72230285


In [4]:
#download geneHancer interactions database by each chromosome from the UCSC genome browser https://genome.ucsc.edu/cgi-bin/hgTables?hgsid=1618294719_EUAaPgOOQPe62s9WPXgbKrGN1JhH&clade=mammal&org=&db=hg38&hgta_group=regulation&hgta_track=geneHancer&hgta_table=geneHancerInteractionsDoubleElite&hgta_regionType=range&position=&hgta_outputType=primaryTable&hgta_outFileName=
load_genehancerInter <- function() {
    list.files(path = '/diskmnt/Projects/snATAC_analysis/GeneHancer/GeneHancerInteractionsDoubleElite', pattern = 'geneHancer*', all.files = T, full.names = T) %>%
    lapply(fread) %>% rbindlist
}

filter <- dplyr::filter
select <- dplyr::select


In [6]:
#Gene Hancer interactions
genehancerInter <- load_genehancerInter()

genehancerInter.gr <- makeGRangesFromDataFrame(genehancerInter[, 1:13],
                         keep.extra.columns=TRUE,
                         ignore.strand=FALSE,
                         seqinfo=NULL,
                         seqnames.field=c('geneHancerChrom'),
                         start.field="geneHancerStart",
                         end.field=c("geneHancerEnd"),
                         strand.field="geneHancerStrand",
                         starts.in.df.are.0based=TRUE)

genehancerInter.dt <- as.data.table(genehancerInter.gr)

# all interactions no filtering
genehancerInter.overlaps <- findOverlaps(subject = StringToGRanges(peakAnno$new_peak),
                                        query = genehancerInter.gr, minoverlap = 400) %>% data.frame


genehancerInter.dt$Genehancer.overlaps.with.peak <- 'NA'
genehancerInter.dt[genehancerInter.overlaps$queryHits, 'Genehancer.overlaps.with.peak'] <- peakAnno$new_peak[genehancerInter.overlaps$subjectHits]

# add overlapping peak info 
genehancerInter <- genehancerInter %>% 
    merge(genehancerInter.dt[,c('name', 'Genehancer.overlaps.with.peak')], by = 'name', all.x = T) %>% 
    mutate(Interacting.pair.geneHancer = paste(Genehancer.overlaps.with.peak, geneName, sep = ':') ,
          Enhancer_gene_pair = paste(geneHancerIdentifier, geneName, sep = ':')) 



## annotate links by variaous annotation files 

In [7]:
cancers = c('CRC', 'BRCA', 'CESC', 'OV', 'PDAC', 'UCEC', 'HNSCC', 'SKCM')
all.links <- fread('/diskmnt/Projects/snATAC_analysis/tumor_Alla/linkingGenes_v7.0/annotate.links/tumor_only_multiome/All_links_tumor_only_multiome_8_cancers.tsv', header = TRUE)

all.links <- all.links %>% 
    merge(peakAnno[,c('SYMBOL', 'new_peak', 'peak.position')], by.x = 'peak', by.y = 'new_peak', all.x = TRUE)

all.links <- all.links %>% dplyr::rename(peak.nearest.gene = SYMBOL)

all.links <- merge(all.links, genehancer[,1:4], by.x = 'peak', by.y = 'Peak', 
                   all.x = T, allow.cartesian=TRUE)

all.links <- merge(all.links, epimap[,1:4], 
                   by.x = 'peak', by.y = 'Peak', all.x = T)

all.links <- merge(all.links, encode[,1:5], 
                   by.x = 'peak', by.y = 'Peak', all.x = T)

all.links <- all.links %>% 
    mutate(Enhancer_gene_pair = paste(GeneHancer.Accession, gene, sep = ':'),
           Enhancer_gene_pair.in.genehancerInter = Enhancer_gene_pair %in% genehancerInter$Enhancer_gene_pair)

#fwrite(all.links, 'Pancan_all.links_annotated_by_GeneHancer_EpiMap_ENCODE.txt', sep ='\t', row.names = F)



In [8]:
all.links %>% filter(gene=='ESR1' & Cancer=='BRCA')


peak,seqnames,start,end,width,strand,score,gene,zscore,pvalue,⋯,GeneHancer.Annotation,EpiMap.Location,EpiMap.Annotation,EpiMap.CRE.width,ENCODE.Location,ENCODE.Accession,ENCODE.Annotation,ENCODE.Annotation_explained,Enhancer_gene_pair,Enhancer_gene_pair.in.genehancerInter
<chr>,<chr>,<int>,<int>,<int>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
chr6-151359387-151359887,chr6,151359637,151656691,297055,*,-0.06711269,ESR1,-2.119057,1.704284e-02,⋯,NA,chr6-151359577-151359757,Enhancer,180,chr6-151359505-151359852,EH38E3741194,dELS,Distal enhancer-like cCRE,NA:ESR1,FALSE
chr6-151390342-151390842,chr6,151390592,151656691,266100,*,0.17294466,ESR1,3.247872,5.813581e-04,⋯,Promoter/Enhancer,chr6-151390340-151390680,Promoter,340,chr6-151390331-151390681,EH38E3741220,"pELS,CTCF-bound","Proximal enhancer-like cCRE, CTCF-bound",GH06J151386:ESR1,FALSE
chr6-151404746-151405246,chr6,151404996,151656691,251696,*,0.05398122,ESR1,1.740887,4.085173e-02,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA:ESR1,FALSE
chr6-151425504-151426004,chr6,151425754,151656691,230938,*,-0.07380408,ESR1,-2.497034,6.261845e-03,⋯,NA,chr6-151425606-151425840,Enhancer,234,chr6-151425580-151425927,EH38E3741238,dELS,Distal enhancer-like cCRE,NA:ESR1,FALSE
chr6-151435741-151436241,chr6,151435991,151656691,220701,*,0.10589405,ESR1,3.548321,1.938478e-04,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA:ESR1,FALSE
chr6-151436662-151437162,chr6,151436912,151656691,219780,*,0.07578552,ESR1,2.314619,1.031689e-02,⋯,Enhancer,chr6-151436820-151437089,Enhancer,269,chr6-151436792-151437122,EH38E3741243,dELS,Distal enhancer-like cCRE,GH06J151436:ESR1,FALSE
chr6-151436662-151437162,chr6,151436912,151656691,219780,*,0.07578552,ESR1,2.314619,1.031689e-02,⋯,Enhancer,chr6-151436990-151437200,Enhancer,210,chr6-151436792-151437122,EH38E3741243,dELS,Distal enhancer-like cCRE,GH06J151436:ESR1,FALSE
chr6-151437399-151437899,chr6,151437649,151656691,219043,*,0.05698549,ESR1,1.784805,3.714649e-02,⋯,Enhancer,chr6-151437519-151437683,Enhancer,164,chr6-151437488-151437744,EH38E2515689,dELS,Distal enhancer-like cCRE,GH06J151436:ESR1,FALSE
chr6-151451941-151452441,chr6,151452191,151656691,204501,*,0.15336652,ESR1,4.653663,1.630447e-06,⋯,Promoter/Enhancer,chr6-151451776-151452820,Promoter,1044,chr6-151452095-151452445,EH38E3741259,"PLS,CTCF-bound","Promoter-like cCRE, CTCF-bound",GH06J151450:ESR1,FALSE


## Diffused links 

In [9]:
#use the folder where the output of Compute_diffuse_links_to_genes_v7.0.R script is saved
cancers = c('BRCA', 'CRC', 'HNSCC', 'CESC', 'OV', 'PDAC', 'UCEC', 'SKCM')

diffused.links <- cancers %>% map(function(c) {
    obj <- readRDS(paste0('/diskmnt/Projects/snATAC_analysis/tumor_Alla/linkingGenes_v7.0/diffuse_links_with_100kb_bins/tumor_only_multiome/out/',c,'_atac.tumor_cells_multiome_obj.20230121.diffuse.links.only.rds'))
    links.tb <- obj %>% as.data.table() %>% mutate(Cancer = c)
    return(links.tb)
}) %>% rbindlist()




In [10]:
# overlap diffused regions with peaks
linked.peaks <- all.links$peak %>% unique()
linked.100kb <- diffused.links$peak %>% unique
overlaps <- findOverlaps(StringToGRanges(linked.peaks), StringToGRanges(linked.100kb), minoverlap = 250) %>% 
    as.data.frame()
overlapping.peaks <- data.frame(peak.peak = linked.peaks[overlaps$queryHits], peak.100kb=linked.100kb[overlaps$subjectHits])

head(overlapping.peaks)


,peak.peak,peak.100kb
,<chr>,<chr>
1,chr1-10000818-10001318,chr1-10000001-10100000
2,chr1-100037789-100038289,chr1-100000001-100100000
3,chr1-100123504-100124004,chr1-100100001-100200000
4,chr1-100132758-100133258,chr1-100100001-100200000
5,chr1-100164137-100164637,chr1-100100001-100200000
6,chr1-100188542-100189042,chr1-100100001-100200000


In [11]:
a <- all.links %>% 
    select(score, peak, gene,pvalue, Cancer) %>% 
    rename(score.peak = score, peak.peak = peak, gene.peak = gene,pvalue.peak = pvalue, Cancer.peak = Cancer)
b <- diffused.links %>% 
    select(score, peak,gene,pvalue, Cancer) %>% 
    rename(score.100kb = score, peak.100kb = peak, gene.100kb = gene,pvalue.100kb = pvalue, Cancer.100kb = Cancer)

tb.for.filtering <- merge(overlapping.peaks, a, by='peak.peak', all = TRUE) %>% 
    merge(b, by='peak.100kb', all = TRUE)



In [12]:
head(tb.for.filtering)

,peak.100kb,peak.peak,score.peak,gene.peak,pvalue.peak,Cancer.peak,score.100kb,gene.100kb,pvalue.100kb,Cancer.100kb
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
1,chr1-1-100000,NA,NA,NA,NA,NA,-0.004316325,AP006222.2,0.0338388421,PDAC
2,chr1-100000001-100100000,chr1-100037789-100038289,0.05132358,CDC14A,0.009562894,BRCA,0.008067725,AGL,0.0271253803,HNSCC
3,chr1-100000001-100100000,chr1-100037789-100038289,0.05132358,CDC14A,0.009562894,BRCA,0.015574480,RTCA,0.0432640297,UCEC
4,chr1-100000001-100100000,chr1-100037789-100038289,0.05132358,CDC14A,0.009562894,BRCA,0.018121272,CDC14A,0.0436024786,PDAC
5,chr1-100000001-100100000,chr1-100037789-100038289,0.05132358,CDC14A,0.009562894,BRCA,0.027838823,FRRS1,0.0167607384,PDAC
6,chr1-100000001-100100000,chr1-100037789-100038289,0.05132358,CDC14A,0.009562894,BRCA,0.041258781,AGL,0.0004101447,CRC


In [13]:
tb.for.filtering <- tb.for.filtering %>% group_by(Cancer.peak) %>% 
    mutate(zscore.peak = scale(score.peak),
            zscore.100kb = scale(score.100kb))


In [14]:
tb.for.filtering <- tb.for.filtering %>% 
    filter((gene.peak==gene.100kb & Cancer.peak==Cancer.100kb) | is.na(peak.100kb)) %>%
    group_by(Cancer.peak) %>% 
    mutate(zscore.peak = scale(score.peak),
            zscore.100kb = scale(score.100kb))

signif.links.by.zscore <- tb.for.filtering %>% filter(zscore.100kb < zscore.peak | is.na(peak.100kb)) %>% 
    mutate(score.dif = case_when(!is.na(score.100kb) ~  zscore.peak - zscore.100kb,
                                TRUE ~ zscore.peak))
           

Warning message:
"Using one column matrices in `filter()` was deprecated in dplyr
1.1.0.
ℹ Please use one dimensional logical vectors instead.
ℹ The deprecated feature was likely used in the dplyr package.
  Please report the issue at
  <https://github.com/tidyverse/dplyr/issues>."


In [15]:
all.links.filtered <- cancers %>% map(function(x) {
    all.links.f <- all.links %>% filter(Cancer==x) %>% 
        mutate(Peak.to.gene = paste(peak, gene, sep = ':'))
    signif.links.f <- signif.links.by.zscore %>% filter(Cancer.peak==x) %>% 
        mutate(Peak.to.gene = paste(peak.peak, gene.peak, sep = ':'))
    toreturn <- all.links.f %>% filter(Peak.to.gene %in% signif.links.f$Peak.to.gene)
    return(toreturn)
}) %>% rbindlist()



In [16]:

all.links.filtered %>% filter(gene=='ESR1' & Cancer=='BRCA')


peak,seqnames,start,end,width,strand,score,gene,zscore,pvalue,⋯,EpiMap.Location,EpiMap.Annotation,EpiMap.CRE.width,ENCODE.Location,ENCODE.Accession,ENCODE.Annotation,ENCODE.Annotation_explained,Enhancer_gene_pair,Enhancer_gene_pair.in.genehancerInter,Peak.to.gene
<chr>,<chr>,<int>,<int>,<int>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
chr6-151435741-151436241,chr6,151435991,151656691,220701,*,0.10589405,ESR1,3.548321,1.938478e-04,⋯,NA,NA,NA,NA,NA,NA,NA,NA:ESR1,FALSE,chr6-151435741-151436241:ESR1
chr6-151451941-151452441,chr6,151452191,151656691,204501,*,0.15336652,ESR1,4.653663,1.630447e-06,⋯,chr6-151451776-151452820,Promoter,1044,chr6-151452095-151452445,EH38E3741259,"PLS,CTCF-bound","Promoter-like cCRE, CTCF-bound",GH06J151450:ESR1,FALSE,chr6-151451941-151452441:ESR1
chr6-151451941-151452441,chr6,151452191,151656691,204501,*,0.15336652,ESR1,4.653663,1.630447e-06,⋯,chr6-151452100-151452420,Promoter,320,chr6-151452095-151452445,EH38E3741259,"PLS,CTCF-bound","Promoter-like cCRE, CTCF-bound",GH06J151450:ESR1,FALSE,chr6-151451941-151452441:ESR1
chr6-151493740-151494240,chr6,151493990,151656691,162702,*,0.14620032,ESR1,2.849093,2.192202e-03,⋯,chr6-151493710-151493900,Promoter,190,chr6-151493899-151494248,EH38E3741286,"PLS,CTCF-bound","Promoter-like cCRE, CTCF-bound",GH06J151492:ESR1,FALSE,chr6-151493740-151494240:ESR1
chr6-151493740-151494240,chr6,151493990,151656691,162702,*,0.14620032,ESR1,2.849093,2.192202e-03,⋯,chr6-151493931-151494167,Promoter,236,chr6-151493899-151494248,EH38E3741286,"PLS,CTCF-bound","Promoter-like cCRE, CTCF-bound",GH06J151492:ESR1,FALSE,chr6-151493740-151494240:ESR1
chr6-151493740-151494240,chr6,151493990,151656691,162702,*,0.14620032,ESR1,2.849093,2.192202e-03,⋯,chr6-151494133-151494240,Promoter,107,chr6-151493899-151494248,EH38E3741286,"PLS,CTCF-bound","Promoter-like cCRE, CTCF-bound",GH06J151492:ESR1,FALSE,chr6-151493740-151494240:ESR1
chr6-151494505-151495005,chr6,151494755,151656691,161937,*,0.13580790,ESR1,5.412243,3.112012e-08,⋯,chr6-151494653-151494868,Promoter,215,chr6-151494587-151494925,EH38E3741288,"pELS,CTCF-bound","Proximal enhancer-like cCRE, CTCF-bound",GH06J151492:ESR1,FALSE,chr6-151494505-151495005:ESR1
chr6-151494505-151495005,chr6,151494755,151656691,161937,*,0.13580790,ESR1,5.412243,3.112012e-08,⋯,chr6-151494877-151495032,Promoter,155,chr6-151494587-151494925,EH38E3741288,"pELS,CTCF-bound","Proximal enhancer-like cCRE, CTCF-bound",GH06J151492:ESR1,FALSE,chr6-151494505-151495005:ESR1
chr6-151548182-151548682,chr6,151548432,151656691,108260,*,0.08517923,ESR1,2.592457,4.764649e-03,⋯,NA,NA,NA,NA,NA,NA,NA,NA:ESR1,FALSE,chr6-151548182-151548682:ESR1


# Add infer CNV

In [21]:
cancers = c('BRCA', 'CRC', 'HNSCC', 'CESC', 'OV', 'PDAC', 'UCEC', 'SKCM')

combo.tumor.meta <- cancers %>% map(function(c) {
    fread(paste0('/diskmnt/Projects/snATAC_primary/PanCan_ATAC_data_freeze/v7.0/Multiome/Merged_objects_PanCan_peaks/Cancer_level/out/', c,'_atac.tumor_cells_multiome_obj.20230121_metadata.tsv'), header=TRUE)
}) %>% rbindlist(fill=TRUE) 

combo.tumor.cells <- combo.tumor.meta %>% pull(V1)


In [22]:
#
gene.cnv <- fread('Gene.cnv.cell.count.by.cancer.multiome.samples.tsv')
total.tumor.count <- combo.tumor.meta %>%
 group_by(Cancer) %>% 
    tally(name = 'N.all.tumor.cells')

cnv <- gene.cnv %>% merge(total.tumor.count, by = 'Cancer', all.x = TRUE) %>% 
    mutate(cnv.pct = N.cells.with.cnv/N.all.tumor.cells)

head(cnv)


Cancer,Gene,cnv,N.cells.with.cnv,N.all.tumor.cells,cnv.pct
<chr>,<chr>,<chr>,<int>,<int>,<dbl>
BRCA,SHC2,Gain,783,49909,0.01568855
BRCA,BSG,Gain,2009,49909,0.04025326
BRCA,PTBP1,Gain,3192,49909,0.06395640
BRCA,ARID3A,Gain,2141,49909,0.04289807
BRCA,TMEM259,Gain,3192,49909,0.06395640
BRCA,ABCA7,Gain,1175,49909,0.02354285


In [23]:
#annotate by inferCNV results
all.links.filtered.cnv <- all.links.filtered %>% 
    merge(cnv[,-5], by.x= c('Cancer', 'gene'), by.y = c('Cancer', 'Gene'), all.x = TRUE,allow.cartesian=TRUE)

all.links.filtered.cnv %>% head()


Cancer,gene,peak,seqnames,start,end,width,strand,score,zscore,⋯,ENCODE.Location,ENCODE.Accession,ENCODE.Annotation,ENCODE.Annotation_explained,Enhancer_gene_pair,Enhancer_gene_pair.in.genehancerInter,Peak.to.gene,cnv,N.cells.with.cnv,cnv.pct
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<int>,<dbl>
BRCA,A1CF,chr10-50849652-50850152,chr10,50849902,50885675,35774,*,0.05668379,9.042230,⋯,chr10-50849793-50849981,EH38E2900532,dELS,Distal enhancer-like cCRE,NA:A1CF,FALSE,chr10-50849652-50850152:A1CF,NA,NA,NA
BRCA,A1CF,chr10-50862595-50863095,chr10,50862845,50885675,22831,*,0.10822264,17.177156,⋯,NA,NA,NA,NA,NA:A1CF,FALSE,chr10-50862595-50863095:A1CF,NA,NA,NA
BRCA,A2M,chr12-8642827-8643327,chr12,8643077,9116229,473153,*,0.12892541,8.851513,⋯,NA,NA,NA,NA,NA:A2M,FALSE,chr12-8642827-8643327:A2M,Gain,10488,0.2101425
BRCA,A2M,chr12-8642827-8643327,chr12,8643077,9116229,473153,*,0.12892541,8.851513,⋯,NA,NA,NA,NA,NA:A2M,FALSE,chr12-8642827-8643327:A2M,Loss,6362,0.1274720
BRCA,A2M,chr12-8687697-8688197,chr12,8687947,9116229,428283,*,0.08111414,6.373127,⋯,chr12-8687833-8688168,EH38E1591389,"dELS,CTCF-bound","Distal enhancer-like cCRE, CTCF-bound",GH12J008687:A2M,FALSE,chr12-8687697-8688197:A2M,Gain,10488,0.2101425
BRCA,A2M,chr12-8687697-8688197,chr12,8687947,9116229,428283,*,0.08111414,6.373127,⋯,chr12-8687833-8688168,EH38E1591389,"dELS,CTCF-bound","Distal enhancer-like cCRE, CTCF-bound",GH12J008687:A2M,FALSE,chr12-8687697-8688197:A2M,Loss,6362,0.1274720


# Annotate by tumor normal DAPs and DEGs


In [24]:
#last updated on 04/07/2023
dap.normal <- fread('/diskmnt/Projects/Users/nvterekhanova/PanCan_snATAC/Analysis/12.InferCNV/1.CNV_perGene_perSample/20230329_PeaksCNV_annot_ExclMet.CEAD.565/out/DACRs_overlap.CNV_annotAdded.closestFeature.pct.0.05.logFch.0.FDR.0.2.Fch.Upd.CNV.0.25.Filtered.20230405.tsv')
deg.normal <- fread('/diskmnt/Projects/Users/nvterekhanova/PanCan_snATAC/Analysis/12.InferCNV/1.CNV_perGene_perSample/20230329_PeaksCNV_annot_ExclMet.CEAD.565/out/DEGs_overlap.CNV_annotAdded.CNV.0.25.pct.0.05.Fch.0.Filtered.20230329.tsv')

filter <- dplyr::filter
select <- dplyr::select

all.links.filtered.cnv

Cancer,gene,peak,seqnames,start,end,width,strand,score,zscore,⋯,ENCODE.Location,ENCODE.Accession,ENCODE.Annotation,ENCODE.Annotation_explained,Enhancer_gene_pair,Enhancer_gene_pair.in.genehancerInter,Peak.to.gene,cnv,N.cells.with.cnv,cnv.pct
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<int>,<dbl>
BRCA,A1CF,chr10-50849652-50850152,chr10,50849902,50885675,35774,*,0.05668379,9.042230,⋯,chr10-50849793-50849981,EH38E2900532,dELS,Distal enhancer-like cCRE,NA:A1CF,FALSE,chr10-50849652-50850152:A1CF,NA,NA,NA
BRCA,A1CF,chr10-50862595-50863095,chr10,50862845,50885675,22831,*,0.10822264,17.177156,⋯,NA,NA,NA,NA,NA:A1CF,FALSE,chr10-50862595-50863095:A1CF,NA,NA,NA
BRCA,A2M,chr12-8642827-8643327,chr12,8643077,9116229,473153,*,0.12892541,8.851513,⋯,NA,NA,NA,NA,NA:A2M,FALSE,chr12-8642827-8643327:A2M,Gain,10488,0.2101425
BRCA,A2M,chr12-8642827-8643327,chr12,8643077,9116229,473153,*,0.12892541,8.851513,⋯,NA,NA,NA,NA,NA:A2M,FALSE,chr12-8642827-8643327:A2M,Loss,6362,0.1274720
BRCA,A2M,chr12-8687697-8688197,chr12,8687947,9116229,428283,*,0.08111414,6.373127,⋯,chr12-8687833-8688168,EH38E1591389,"dELS,CTCF-bound","Distal enhancer-like cCRE, CTCF-bound",GH12J008687:A2M,FALSE,chr12-8687697-8688197:A2M,Gain,10488,0.2101425
BRCA,A2M,chr12-8687697-8688197,chr12,8687947,9116229,428283,*,0.08111414,6.373127,⋯,chr12-8687833-8688168,EH38E1591389,"dELS,CTCF-bound","Distal enhancer-like cCRE, CTCF-bound",GH12J008687:A2M,FALSE,chr12-8687697-8688197:A2M,Loss,6362,0.1274720
BRCA,A2M,chr12-8697618-8698118,chr12,8697868,9116229,418362,*,0.09995946,3.311654,⋯,chr12-8697791-8698140,EH38E3002285,"PLS,CTCF-bound","Promoter-like cCRE, CTCF-bound",GH12J008695:A2M,FALSE,chr12-8697618-8698118:A2M,Gain,10488,0.2101425
BRCA,A2M,chr12-8697618-8698118,chr12,8697868,9116229,418362,*,0.09995946,3.311654,⋯,chr12-8697791-8698140,EH38E3002285,"PLS,CTCF-bound","Promoter-like cCRE, CTCF-bound",GH12J008695:A2M,FALSE,chr12-8697618-8698118:A2M,Loss,6362,0.1274720
BRCA,A2M,chr12-8723611-8724111,chr12,8723861,9116229,392369,*,0.07505700,4.584384,⋯,NA,NA,NA,NA,NA:A2M,FALSE,chr12-8723611-8724111:A2M,Gain,10488,0.2101425


In [25]:
daps.up.tumor <- dap.normal %>% 
    dplyr::filter(Disease %in% c('SKCM', 'CESC', 'OV', 'UCEC','BRCA','BRCA_Basal', 'PDAC', 'CRC', 'HNSCC')) %>%
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_peak = paste(Cancer, peak, sep ='_')) %>%
    dplyr::filter(avg_log2FC >0.5 & p_val_adj < 0.05) %>% pull(Cancer_peak) %>% unique

daps.down.tumor <- dap.normal %>% 
    dplyr::filter(Disease %in% c('SKCM',  'CESC', 'OV', 'UCEC','BRCA', 'BRCA_Basal', 'PDAC', 'CRC', 'HNSCC')) %>%
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_peak = paste(Cancer, peak, sep ='_')) %>%
    dplyr::filter(avg_log2FC < -0.5 & p_val_adj < 0.05) %>% pull(Cancer_peak) %>% unique

not.daps <- dap.normal %>% 
    dplyr::filter(Disease %in% c('SKCM', 'CESC', 'OV', 'UCEC','BRCA','BRCA_Basal', 'PDAC', 'CRC', 'HNSCC')) %>%
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_peak = paste(Cancer, peak, sep ='_')) %>%
    dplyr::filter(p_val_adj > 0.05) %>% pull(Cancer_peak) %>% unique

length(daps.up.tumor)
length(daps.down.tumor)



[1] 7316

[1] 17851

In [26]:
degs.up.tumor <- deg.normal %>% 
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
                   Cancer_gene = paste(Cancer, Gene, sep ='_')) %>%
    dplyr::filter(avg_log2FC >0.25 & p_val_adj < 0.05) %>% pull(Cancer_gene) %>% unique

degs.down.tumor <- deg.normal %>% 
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_gene = paste(Cancer, Gene, sep ='_')) %>%
    dplyr::filter(avg_log2FC < -0.25 & p_val_adj < 0.05) %>% pull(Cancer_gene) %>% unique

not.degs <- deg.normal %>% 
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_gene = paste(Cancer, Gene, sep ='_')) %>%
    dplyr::filter( p_val_adj > 0.05 | abs(avg_log2FC) < 0.25) %>% pull(Cancer_gene) %>% unique

length(degs.up.tumor)
length(degs.down.tumor)

[1] 3714

[1] 6855

In [29]:
all.links.filtered.cnv.tumor.normal <- all.links.filtered.cnv %>% mutate(
    DAP.tumor = case_when(paste(Cancer, peak, sep='_') %in% daps.up.tumor ~ 'Up in tumor',
                          paste(Cancer, peak, sep='_') %in% daps.down.tumor ~ 'Down in tumor',
                          paste(Cancer, peak, sep='_') %in% not.daps ~ 'Not a DAP',
                          peak %in% peakAnno$new_peak ~ 'Not tested',
                                         TRUE ~ 'NA')) %>%
    mutate(DEG.tumor = case_when(paste(Cancer, gene, sep='_') %in% degs.up.tumor ~ 'Up in tumor',
                          paste(Cancer, gene, sep='_') %in% degs.down.tumor ~ 'Down in tumor',
                          paste(Cancer, gene, sep='_') %in% not.degs ~ 'Not a DEG',
                                         TRUE ~ 'Not tested'))


In [30]:
all.links.filtered.cnv.tumor.normal %>% head

Cancer,gene,peak,seqnames,start,end,width,strand,score,zscore,⋯,ENCODE.Annotation,ENCODE.Annotation_explained,Enhancer_gene_pair,Enhancer_gene_pair.in.genehancerInter,Peak.to.gene,cnv,N.cells.with.cnv,cnv.pct,DAP.tumor,DEG.tumor
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>
BRCA,A1CF,chr10-50849652-50850152,chr10,50849902,50885675,35774,*,0.05668379,9.042230,⋯,dELS,Distal enhancer-like cCRE,NA:A1CF,FALSE,chr10-50849652-50850152:A1CF,NA,NA,NA,Not tested,Not tested
BRCA,A1CF,chr10-50862595-50863095,chr10,50862845,50885675,22831,*,0.10822264,17.177156,⋯,NA,NA,NA:A1CF,FALSE,chr10-50862595-50863095:A1CF,NA,NA,NA,Not tested,Not tested
BRCA,A2M,chr12-8642827-8643327,chr12,8643077,9116229,473153,*,0.12892541,8.851513,⋯,NA,NA,NA:A2M,FALSE,chr12-8642827-8643327:A2M,Gain,10488,0.2101425,Not tested,Not tested
BRCA,A2M,chr12-8642827-8643327,chr12,8643077,9116229,473153,*,0.12892541,8.851513,⋯,NA,NA,NA:A2M,FALSE,chr12-8642827-8643327:A2M,Loss,6362,0.1274720,Not tested,Not tested
BRCA,A2M,chr12-8687697-8688197,chr12,8687947,9116229,428283,*,0.08111414,6.373127,⋯,"dELS,CTCF-bound","Distal enhancer-like cCRE, CTCF-bound",GH12J008687:A2M,FALSE,chr12-8687697-8688197:A2M,Gain,10488,0.2101425,Not tested,Not tested
BRCA,A2M,chr12-8687697-8688197,chr12,8687947,9116229,428283,*,0.08111414,6.373127,⋯,"dELS,CTCF-bound","Distal enhancer-like cCRE, CTCF-bound",GH12J008687:A2M,FALSE,chr12-8687697-8688197:A2M,Loss,6362,0.1274720,Not tested,Not tested


# Annotate by met vs primary DAPs and DEGs

In [31]:
#last updated on 04/07/2023
dap.met <- fread('/diskmnt/Projects/Users/nvterekhanova/PanCan_snATAC/Analysis/1.EMT_analysis/Objects_v7/20230303_EMT_DACRs_cohortObj.relaxed.pct/out/DACRs_Met_vs_Primary_Signac.1.8.FoldChange.CohortObj.20230330.tsv')
deg.met <- fread('/diskmnt/Projects/Users/nvterekhanova/PanCan_snATAC/Analysis/1.EMT_analysis/Objects_v7/20230124_EMF_DEGs_MergedObj/out/Met_vs_Primary_DEGs.4Cohorts.20230124.tsv')

filter <- dplyr::filter
select <- dplyr::select

In [32]:
daps.up.met <- dap.met %>% 
    filter(Disease %in% c('SKCM', 'CESC', 'OV', 'UCEC','BRCA','BRCA_Basal', 'PDAC', 'CRC', 'HNSCC')) %>%
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_peak = paste(Cancer, peak, sep ='_')) %>%
    filter(avg_log2FC > 0.5 & p_val_adj < 0.05) %>% pull(Cancer_peak) %>% unique

daps.down.met <- dap.met %>% 
    filter(Disease %in% c('SKCM',  'CESC', 'OV', 'UCEC','BRCA', 'BRCA_Basal', 'PDAC', 'CRC', 'HNSCC')) %>%
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_peak = paste(Cancer, peak, sep ='_')) %>%
    filter(avg_log2FC < -0.5 & p_val_adj < 0.05) %>% pull(Cancer_peak) %>% unique

not.daps <- dap.met %>% 
    filter(Disease %in% c('SKCM', 'CESC', 'OV', 'UCEC','BRCA','BRCA_Basal', 'PDAC', 'CRC', 'HNSCC')) %>%
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_peak = paste(Cancer, peak, sep ='_')) %>%
    filter(p_val_adj > 0.05) %>% pull(Cancer_peak) %>% unique


length(daps.up.met)
length(daps.down.met)



[1] 47550

[1] 107134

In [33]:
degs.up.met <- deg.met %>% 
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
                   Cancer_gene = paste(Cancer, Gene, sep ='_')) %>%
    filter(avg_log2FC >0.25 & p_val_adj < 0.05) %>% pull(Cancer_gene) %>% unique

degs.down.met <- deg.met %>% 
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_gene = paste(Cancer, Gene, sep ='_')) %>%
    filter(avg_log2FC < -0.25 & p_val_adj < 0.05) %>% pull(Cancer_gene) %>% unique

not.degs <- deg.met %>% 
    mutate(Cancer=str_split_fixed(Disease, '_', 2)[,1],
           Cancer_gene = paste(Cancer, Gene, sep ='_')) %>%
    filter( p_val_adj > 0.05) %>% pull(Cancer_gene) %>% unique

length(degs.up.met)
length(degs.down.met)

[1] 2230

[1] 1878

In [34]:

all.links.filtered.cnv.tumor.normal <- all.links.filtered.cnv %>% mutate(
    DAP.met = case_when(paste(Cancer, peak, sep='_') %in% daps.up.met ~ 'Up in mets',
                          paste(Cancer, peak, sep='_') %in% daps.down.met ~ 'Down in mets',
                          paste(Cancer, peak, sep='_') %in% not.daps ~ 'Not a DAP',
                          peak %in% peakAnno$new_peak ~ 'Not tested',
                                         TRUE ~ 'NA')) %>%
    mutate(DEG.met = case_when(paste(Cancer, gene, sep='_') %in% degs.up.met ~ 'Up in mets',
                          paste(Cancer, gene, sep='_') %in% degs.down.met ~ 'Down in mets',
                          paste(Cancer, gene, sep='_') %in% not.degs ~ 'Not a DEG',
                                         TRUE ~ 'Not tested'))
